In [1]:
import pandas as pd
from pandas import Series, DataFrame
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
%matplotlib inline

data = pd.read_csv("C:/Users/SS/Desktop/2019-2nd-ml-month-with-kakr/train.csv")
test = pd.read_csv("C:/Users/SS/Desktop/2019-2nd-ml-month-with-kakr/test.csv")
sample = pd.read_csv("C:/Users/SS/Desktop/2019-2nd-ml-month-with-kakr/sample_submission.csv")

data['log_price'] = sp.log1p(data['price'])

data = data[data.sqft_living != 13540]
data= data[data.id != 2302]
data= data[data.id != 4123]
data= data[data.id != 7173]
data= data[data.id != 2775]

for i in ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement'] : 
      
        data[i] = sp.log1p(data[i])
        test[i] = sp.log1p(test[i])
        
data['date'] = data['date'].apply(lambda x : x[0:8])
test['date'] = test['date'].apply(lambda x : x[0:8])

data['yr_renovated'] = data['yr_renovated'].apply(lambda x : sp.nan if x == 0 else x )
data['yr_renovated'] = data['yr_renovated'].fillna(data['yr_built'])

test['yr_renovated'] = test['yr_renovated'].apply(lambda x : sp.nan if x == 0 else x )
test['yr_renovated'] = test['yr_renovated'].fillna(test['yr_built'])

for df in [data, test] : 
    
    df['total_rooms'] = df['bedrooms'] + df['bathrooms']
    df['sqft_ratio'] = df['sqft_living'] / df['sqft_lot']
    df['sqft_total_size'] = df['sqft_above'] + df['sqft_basement']
    df['sqft_ratio15'] = df['sqft_living15'] / df['sqft_lot15'] 
    df['is_renovated'] = df['yr_renovated'] - df['yr_built']
    df['is_renovated'] = df['is_renovated'].apply(lambda x: 0 if x == 0 else 1)
    df['date'] = df['date'].astype('int')
    
data['per_price'] = data['price']/ data['sqft_total_size']
zipcode_price = data.groupby(['zipcode'])['per_price'].agg({'mean','var'}).reset_index()

data = pd.merge(data, zipcode_price,how='left', on='zipcode')
test = pd.merge(test, zipcode_price,how='left', on='zipcode')

for df in [data, test]:
    
    df['mean'] = df['mean'] * df['sqft_total_size']
    df['var'] = df['var'] * df['sqft_total_size']
    
x_train = data.drop(['id','date','price','log_price','per_price'], axis = 1 )
y_train = data['log_price']
x_test = test.drop(['id','date'], axis = 1)

C:\Users\SS\Anaconda31\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error

cross_val_score(LinearRegression() , x_train , y_train, cv = 5)    # train 가지고 cv돌려서 rmse구하기?

array([0.84476674, 0.8373855 , 0.83153954, 0.84256888, 0.83504258])

In [52]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5)
cross_val_score(LinearRegression(), x_train , y_train , cv =kfold)

array([0.84476674, 0.8373855 , 0.83153954, 0.84256888, 0.83504258])

In [53]:
scores = cross_val_score(LinearRegression(), x_train , y_train , cv =kfold, scoring = 'mean_squared_error')

C:\Users\SS\Anaconda31\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\SS\Anaconda31\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\SS\Anaconda31\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\SS\Anaconda31\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\SS\Anaconda31\lib\site-packages

In [54]:
sp.sqrt(-scores).mean()

0.21144953719727827

In [59]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor


kfold = KFold(n_splits=5)
scores = cross_val_score(RandomForestRegressor(n_estimators = 100), x_train , y_train , cv =kfold, scoring = 'mean_squared_error')
scores
# sp.sqrt(-scores).mean()  # y 변환 안해줌 

C:\Users\SS\Anaconda31\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\SS\Anaconda31\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\SS\Anaconda31\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\SS\Anaconda31\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\SS\Anaconda31\lib\site-packages

array([-0.0318619 , -0.03331063, -0.03389312, -0.03123672, -0.02809682])

In [57]:
x_train

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,long,sqft_living15,sqft_lot15,total_rooms,sqft_ratio,sqft_total_size,sqft_ratio15,is_renovated,mean,var
0,3,1.00,7.074117,8.639588,1.0,0,0,3,7,7.074117,...,-122.257,1340,5650,4.00,0.818803,7.074117,0.237168,0,2.173306e+05,1.242862e+09
1,2,1.00,6.647688,9.210440,1.0,0,0,3,6,6.647688,...,-122.233,2720,8062,3.00,0.721756,6.647688,0.337385,0,3.195676e+05,2.171972e+09
2,3,2.00,7.427144,8.997271,1.0,0,0,3,8,7.427144,...,-122.045,1800,7503,5.00,0.825489,7.427144,0.239904,0,5.804092e+05,6.788666e+09
3,3,2.25,7.447751,8.827615,2.0,0,0,3,7,7.447751,...,-122.327,2238,6819,5.25,0.843688,7.447751,0.328201,0,2.373334e+05,8.954380e+08
4,3,1.50,6.966967,9.181118,1.0,0,0,3,7,6.966967,...,-122.315,1650,9711,4.50,0.758837,6.966967,0.169910,0,2.300040e+05,1.524919e+09
5,3,2.50,8.177797,9.189831,1.0,0,0,3,8,7.528869,...,-122.145,2210,8925,5.50,0.889874,14.967841,0.247619,0,1.051882e+06,1.078127e+10
6,2,1.00,7.057037,8.699681,1.0,0,0,4,7,6.758095,...,-122.292,1330,6000,3.00,0.811183,12.465205,0.221667,0,7.114793e+05,5.102226e+09
7,3,1.00,7.266129,9.898576,1.5,0,0,4,7,7.266129,...,-122.229,1780,12697,4.00,0.734058,7.266129,0.140191,0,3.492973e+05,2.374033e+09
8,3,1.75,7.223296,9.177920,1.0,0,0,4,7,7.223296,...,-122.045,1370,10208,4.75,0.787030,7.223296,0.134208,0,5.644791e+05,6.602341e+09
9,5,2.00,7.501634,8.486940,1.5,0,0,3,7,7.501634,...,-122.394,1360,4850,7.00,0.883903,7.501634,0.280412,0,4.694029e+05,6.635195e+09


In [58]:
y_train

0        12.309987
1        12.100718
2        13.142168
3        12.458779
4        12.583999
5        13.403777
6        13.056226
7        12.644331
8        12.899222
9        13.180634
10       13.384729
11       12.886644
12       13.091906
13       12.149508
14       12.345839
15       12.861001
16       14.508658
17       12.439962
18       12.703816
19       12.358798
20       13.750440
21       13.410547
22       13.271647
23       13.440819
24       13.190024
25       12.683862
26       13.453106
27       13.369225
28       12.388398
29       13.312985
           ...    
15000    12.706851
15001    13.318754
15002    13.573440
15003    13.322772
15004    12.899097
15005    13.269059
15006    12.800783
15007    13.377007
15008    13.296093
15009    13.102163
15010    12.899097
15011    12.506181
15012    13.132316
15013    12.861001
15014    12.934831
15015    12.758523
15016    14.016409
15017    13.899853
15018    12.765691
15019    13.429776
15020    14.269766
15021    13.

In [65]:
import pandas as pd
from pandas import Series, DataFrame
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
%matplotlib inline

data = pd.read_csv("C:/Users/SS/Desktop/2019-2nd-ml-month-with-kakr/train.csv")
test = pd.read_csv("C:/Users/SS/Desktop/2019-2nd-ml-month-with-kakr/test.csv")
sample = pd.read_csv("C:/Users/SS/Desktop/2019-2nd-ml-month-with-kakr/sample_submission.csv")

data['log_price'] = sp.log1p(data['price'])

data = data[data.sqft_living != 13540]
data= data[data.id != 2302]
data= data[data.id != 4123]
data= data[data.id != 7173]
data= data[data.id != 2775]

for i in ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement'] : 
      
        data[i] = sp.log1p(data[i])
        test[i] = sp.log1p(test[i])
        
data['date'] = data['date'].apply(lambda x : x[0:8])
test['date'] = test['date'].apply(lambda x : x[0:8])

data['yr_renovated'] = data['yr_renovated'].apply(lambda x : sp.nan if x == 0 else x )
data['yr_renovated'] = data['yr_renovated'].fillna(data['yr_built'])

test['yr_renovated'] = test['yr_renovated'].apply(lambda x : sp.nan if x == 0 else x )
test['yr_renovated'] = test['yr_renovated'].fillna(test['yr_built'])

for df in [data, test] : 
    
    df['total_rooms'] = df['bedrooms'] + df['bathrooms']
    df['sqft_ratio'] = df['sqft_living'] / df['sqft_lot']
    df['sqft_total_size'] = df['sqft_above'] + df['sqft_basement']
    df['sqft_ratio15'] = df['sqft_living15'] / df['sqft_lot15'] 
    df['is_renovated'] = df['yr_renovated'] - df['yr_built']
    df['is_renovated'] = df['is_renovated'].apply(lambda x: 0 if x == 0 else 1)
    df['date'] = df['date'].astype('int')
    
data['per_price'] = data['price']/ data['sqft_total_size']
zipcode_price = data.groupby(['zipcode'])['per_price'].agg({'mean','var'}).reset_index()

data = pd.merge(data, zipcode_price,how='left', on='zipcode')
test = pd.merge(test, zipcode_price,how='left', on='zipcode')

for df in [data, test]:
    
    df['mean'] = df['mean'] * df['sqft_total_size']
    df['var'] = df['var'] * df['sqft_total_size']
    
x_train = data.drop(['id','date','price','log_price','per_price'], axis = 1 )
y_train = data['price']
x_test = test.drop(['id','date'], axis = 1)

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor


kfold = KFold(n_splits=5)
scores = cross_val_score(RandomForestRegressor(n_estimators = 100), x_train , y_train , cv =kfold, scoring = 'neg_mean_squared_error')

In [63]:
sp.sqrt(-scores).mean()

131290.0344647036

In [64]:
x_train = data.drop(['id','date','price','log_price','per_price'], axis = 1 )
y_train = data['log_price']
x_test = test.drop(['id','date'], axis = 1)

ran = RandomForestRegressor(n_estimators = 100).fit(x_train.values, y_train.values)
ran.predict(x_test.values)
pred = sp.exp(ran.predict(x_test.values)) - 1 
sample['price'] = pred
sample.to_csv("C:/Users/SS/Desktop/2019-2nd-ml-month-with-kakr/sampleranfo.csv", index = False)

In [ ]:
-2.11361358e+10, -1.88994450e+10, -1.47820929e+10, -1.60494754e+10,
       -1.57061159e+10])

In [66]:
scores

array([-2.12155817e+10, -1.86226424e+10, -1.47992943e+10, -1.57775833e+10,
       -1.55820400e+10])